In [20]:
import nltk
import numpy as np
import random
import string
import os.path
from os import path

In [12]:
f=open('/Users/amipatel/Desktop/kb.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()
raw=' '.join(raw.split())

sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [13]:
sent_tokens[:2]

['people tend to lose weight quickly on low carbohydrate diets because they restrict their calories to about 1,200 a day.',
 "this is about 500 calories per day for women, and 600 for men.you can choose whichever two days of the week you prefer, as long as there is at least one non fasting day in between them.one common way of planning the week is to fast on mondays and thursdays, with two or three small meals, then eat normally for the rest of the week.it's important to emphasize that eating normally does not mean you can eat anything."]

In [14]:
inputs = ("hello", "hi", "greetings", "hey")
response = ["Hey, let's start with your name. Please introduce yourself"]

def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in inputs:
            return response

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
from textblob import TextBlob
import re

In [17]:
simpleChat = [['PRP','VBP', 'DT', 'JJ', 'NN'], 
              ['PRP','VBP','DT', 'NN'], 
              ['PRP','VBP', 'JJ'], 
              ['PRP','VBP', 'JJ', 'NN']] 

def find_prp(s):
    response = None
    
    for w,POS in s.pos_tags:
        if w == 'you':
            response = 'I'
        if w == 'i':
            response = 'You'
            
    return response

def find_nn(s):
    nn = None
    
    for w,POS in s.pos_tags:
        if 'NN' in POS:
            nn = w
            
    return nn
    
def find_vp(s):
    vb = None
    
    for w,POS in s.pos_tags:
        if 'VB' in POS:
            if w == 'am':
                vb = 'are'
            if w == 'are':
                vb = 'am'
            else:
                vb = w
            
    return vb

def find_adj(s):
    adj = None
    
    for w,POS in s.pos_tags:
        if 'JJ' in POS:
            adj = w
            
    return adj

def find_dt(s):
    dt = None
    
    for w,POS in s.pos_tags:
        if 'DT' in POS:
            dt = w
            
    return dt

def find_parts_of_speech(parsed):
    for s in parsed.sentences:
        prp_response = find_prp(s)
        nn_response = find_nn(s)
        vb_response = find_vp(s)
        adj_response = find_adj(s)
        dt_response = find_dt(s)
    return prp_response, nn_response, vb_response, adj_response, dt_response
    
def response2(user):
    sent_tokens.append(user)
    user = user.lower()
    punctuations = '''!()-[]{};:"\,<>/@#$%^&*_~'''
    for x in user: 
        if x in punctuations: 
            user = user.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .?]+', ' ',user)
    parsed = TextBlob(content)
    prp, nn, vb, adj, dt = find_parts_of_speech(parsed)
    pos = []
    for w,POS in parsed.pos_tags:
        pos.append(POS)
    res = ''
    if '?' in user:
        return response(user)
    elif pos in simpleChat:
        res = 'Sure, '
        if parsed.noun_phrases is not []:
            if nn is not None:
                if dt is None and adj is not None:
                    res += prp + ' ' + vb + ' ' + adj + ' ' + nn + '!'
                elif dt is None and adj is None:
                    res += prp + ' ' + vb + ' ' + nn + '!'
                elif dt is not None and adj is None:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + nn + '!'
                else:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + adj + ' ' + nn + '!'
            else:
                if dt is None and adj is not None:
                    res += prp + ' ' + vb + ' ' + adj + '!'
                elif dt is None and adj is None:
                    res += prp + ' ' + vb + '!'
                elif dt is not None and adj is None:
                    res += prp + ' ' + vb + ' ' + dt + '!'
                else:
                    res += prp + ' ' + vb + ' ' + dt + ' ' + adj + '!'
        elif adj is not None:
            res += prp + ' ' + vb + ' ' + adj + '!'
        else:
            if dt is not None and nn is not None:
                res += prp + ' ' + vb + ' ' + dt + ' ' + nn + '!'
        sent_tokens.remove(user)
        return res
    else:
        return response(user)
    

In [33]:
def response(user_response):
    bot_response=''
    user_response = user_response.lower()
    punctuations = '''!()-[]{};:"\,<>/@#$%^&*_~'''
    for x in user_response: 
        if x in punctuations: 
            user_response = user_response.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .?]+', ' ',user_response)
    question = False
    use = ''    
    if '?' in content:
        if(content=='how are you?' or content=='what is your name?'):  
            if content=='how are you?':
                bot_response = 'I am fine!'
            else:
                bot_response = 'My name is DIETO. I will answer your queries about diets.'
            sent_tokens.remove(user_response)
            return bot_response
        elif 'diet' in content:
            parse = TextBlob(content)
            if ('what', 'WP') in parse.pos_tags:
                question = True
                for w,POS in parse.pos_tags:
                    if 'WP' in POS:
                        continue
                    else:
#                         if w is not 'diet':
#                             use += w
#                         elif POS == 'JJ':
                        use += w
                        
            sent_tokens.append(use)   
   
    TfidfVec = TfidfVectorizer()
    tfidf = TfidfVec.fit_transform(sent_tokens)
    similar_vector_vals = cosine_similarity(tfidf[-1], tfidf)
    
    # the second last sentence will have the highest cosine similarity
    if question is True:
        sentence_number=similar_vector_vals.argsort()[0][-3]  
    else:
        sentence_number=similar_vector_vals.argsort()[0][-2]
    
    # Flatten to check if the cosine similarity is 0 or not
    flat = similar_vector_vals.flatten()
    flat.sort()
    actual_tfidf = flat[-2]
    if(actual_tfidf==0):
        bot_response=bot_response+"I am sorry! I don't understand you"
        sent_tokens.remove(user_response)
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[sentence_number]
        if question is True:
            sent_tokens.remove(use)
        sent_tokens.remove(user_response)
        return bot_response

In [45]:
Bye = False
print("DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!")
file = 'users.txt'
while(Bye is not True):
    print("YOU: ", end = '')
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            print("DIETO: You are welcome.")
        else:
            if(greeting(user_response)!=None):
                print("DIETO: " + greeting(user_response))
                print("YOU: ", end = '')
                user_response = input()
                if(user_response!='bye'):
                    p = TextBlob(user_response)
                    names = []
                    if path.exists(file):
                        f = open(file, 'r') 
                        names = f.read().split('\n') 
                        f.close()
                    new_name = p.noun_phrases
                    if new_name[0] not in names:
                        with open(file, 'a') as f:
                            f.write(new_name[0] + '\n')
                        f.close()
                        print("DIETO: Nice to meet you, " + new_name[0] + "!")
                    else:
                        print("DIETO: Welcome back, " + new_name[0] + "!")
                else:
                    Bye = True
                    print("DIETO: Bye! take care..")  
            else:
                print("DIETO: ",end="")
                print(response2(user_response))
    else:
        Bye = True
        print("DIETO: Bye! take care..")

DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!
YOU: Hi
DIETO: Hey, let's start with your name. Please introduce yourself
YOU: My name is Ami
DIETO: Welcome back, ami!
YOU: how much food do you recommend?
DIETO: as far as weight loss goes, says heber, the diet will do nothing for you long term, since the change in how much you eat lasts only as long as you use the feeding tube.
YOU: name any type of diet?
DIETO: I am sorry! I don't understand you
YOU: Do you recommend ketosis?
DIETO: and we get residues of the hormones in those foods, so if you are going to eat, i recommend reducing animal foods in the diet.
YOU: How much calories should i consume?
DIETO: experts recommend that healthy adult women consume about 2,000 calories and man consume 2,500 calories daily.
YOU: bye
DIETO: Bye! take care..


In [46]:
text = input()
p = TextBlob(text)
print(p.pos_tags)
p.noun_phrases


My name is Ami
[('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Ami', 'NNP')]


WordList(['ami'])